In [ ]:
import numpy as np
import pandas as pd
import os
import random
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split

**Data Preprocessing**

In [ ]:
data_path = '/kaggle/input/satellite-image-classification/data'
os.listdir(data_path)

In [ ]:
#Make a directory with image paths and corresponding labels
data_df = pd.DataFrame(columns = ['Image_Path', 'label'])


for folder in os.listdir(data_path):
    folder_path = os.path.join(data_path, folder)
    for image in os.listdir(folder_path):
        image_path = os.path.join(folder_path, image)
        label = folder
        data_df = data_df._append({'Image_Path': image_path, 'label':label}, ignore_index = True)
        
data_df.head()

In [ ]:
indices = random.sample(range(5600), 9)

# Define the figure size and layout
fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(8,8))

# Loop over the axes and plot the corresponding images
for i, ax in enumerate(axes.flat):
    # Get the image and label for the current index
    img = plt.imread(data_df.iloc[indices[i]].Image_Path)
    label = data_df.iloc[indices[i]].label
    
    # Plot the image and label
    ax.imshow(img)
    ax.set_title(label)
    ax.axis('off')

# Show the plot
plt.show()

In [ ]:
data_df['label'].value_counts()

**Split the Data**

In [ ]:
train, val_test = train_test_split(data_df, test_size = 0.2, random_state = 42)
val, test = train_test_split(val_test, test_size = 0.5, random_state = 42)


In [ ]:
print("No of training samples : ", train.shape[0])
print("No of validation samples : ", val.shape[0])
print("No of testing samples : ", test.shape[0])

In [ ]:
train['label'].value_counts()

In [ ]:

train_datagen = ImageDataGenerator(rescale=1./255,
                                      rotation_range=180,
                                      width_shift_range=0.2,
                                      height_shift_range=0.2,
                                      shear_range = 0.2,
                                      zoom_range=0.2,
                                      horizontal_flip=True,
                                      fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = train_datagen.flow_from_dataframe(dataframe = train,
                                                   x_col = 'Image_Path',
                                                   y_col = 'label',
                                                   target_size = (100,100),
                                                   #batch_size = 32,
                                                   class_mode = 'categorical')

test_generator = test_datagen.flow_from_dataframe(dataframe = test,
                                                 x_col = 'Image_Path',
                                                 y_col = 'label',
                                                 target_size = (100,100),
                                                 #batch_size = 32,
                                                 class_mode = 'categorical')

val_generator = test_datagen.flow_from_dataframe(dataframe = val,
                                                x_col = 'Image_Path',
                                                y_col = 'label',
                                                target_size = (100,100),
                                                #batch_size = 32,
                                                class_mode = 'categorical')

**CNN Model**

In [ ]:
model = Sequential()

model.add(Conv2D(32, (3,3), activation = 'relu', input_shape = (100,100,3)))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Conv2D(64, (3,3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Conv2D(128, (3,3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Flatten())
model.add(Dropout(0.1))

model.add(Dense(512, activation = 'relu'))
model.add(Dropout(0.1))


model.add(Dense(4, activation = 'softmax'))

model.summary()

In [ ]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
history = model.fit(train_generator, epochs = 11, validation_data = val_generator)

In [ ]:
score = model.evaluate(test_generator)

In [ ]:
plt.plot(history.history['accuracy'], label = 'Training_Accuracy')
plt.plot(history.history['val_accuracy'], label = 'Validation_Accuracy')
plt.xlabel('Epochs')
plt.title("Model Accuracy")
plt.legend()
plt.show()

plt.plot(history.history['loss'], label = 'Training_Loss')
plt.plot(history.history['val_loss'], label = 'Validation Loss')
plt.xlabel("Epochs")
plt.title("Model Loss")
plt.legend()
plt.show()

**Transfer Learning**

In [ ]:
import tensorflow_hub as hub
from tensorflow.keras import layers

In [ ]:
resnet_url = "https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/4"

In [ ]:
feature_extraction_layer = hub.KerasLayer(resnet_url, 
                                         trainable = False,
                                         name = 'feature_extraction_layer',
                                         input_shape = (224,224,3))

model = tf.keras.Sequential([
    feature_extraction_layer,
    layers.Dense(4, activation = 'softmax', name = 'output_layer')
])

model.summary()

In [ ]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
history = model.fit(train_generator, epochs = 11, validation_data = val_generator)

In [ ]:
model.evaluate(test_generator)

In [ ]:
plt.plot(history.history['accuracy'], label = 'Training_Accuracy')
plt.plot(history.history['val_accuracy'], label = 'Validation_Accuracy')
plt.xlabel('Epochs')
plt.title("Model Accuracy")
plt.legend()
plt.show()

plt.plot(history.history['loss'], label = 'Training_Loss')
plt.plot(history.history['val_loss'], label = 'Validation Loss')
plt.xlabel("Epochs")
plt.title("Model Loss")
plt.legend()
plt.show()

In [ ]:
import cv2
from skimage.transform import resize
from skimage.io import imread

def predict_new(imgpath, model):
      image = plt.imread(imgpath)
      plt.imshow(image)
      image_resized = resize(image, (224,224,3))
      image = image_resized.reshape(1,224,224,3)
      pred = model.predict(image)
      print(pred)
      x = np.argmax(pred)
      if x == 0:
            print("Cloudy")
      elif x == 1:
        print("Desert")
      elif x == 2:
        print("Green Area")
      else:
        print("Water")



In [ ]:
cloudy = '/kaggle/input/satellite-image-classification/data/cloudy/train_10598.jpg'
desert = '/kaggle/input/satellite-image-classification/data/desert/desert(1002).jpg'
green_area = '/kaggle/input/satellite-image-classification/data/green_area/Forest_1465.jpg'
water = '/kaggle/input/satellite-image-classification/data/water/SeaLake_1048.jpg'


In [ ]:
predict_new(cloudy, model)

In [ ]:
predict_new(desert, model)

In [ ]:
predict_new(green_area, model)

In [ ]:
predict_new(water, model)